In [1]:
%pip install pyreadstat --quiet

Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import pandas as pd
df = pd.read_spss('CCAM SPSS Data 2008-2022.sav')

In [3]:
df.columns

Index(['case_ID', 'wave', 'year', 'weight_wave', 'weight_aggregate',
       'happening', 'cause_original', 'cause_other_text', 'cause_recoded',
       'sci_consensus', 'worry', 'harm_personally', 'harm_US',
       'harm_dev_countries', 'harm_future_gen', 'harm_plants_animals',
       'when_harm_US', 'reg_CO2_pollutant', 'reg_utilities', 'fund_research',
       'reg_coal_emissions', 'discuss_GW', 'hear_GW_media', 'gender', 'age',
       'age_category', 'generation', 'educ', 'educ_category', 'income',
       'income_category', 'race', 'ideology', 'party', 'party_w_leaners',
       'party_x_ideo', 'registered_voter', 'region9', 'region4', 'religion',
       'religion_other_nonchristian', 'evangelical', 'service_attendance',
       'marit_status', 'employment', 'house_head', 'house_size',
       'house_ages0to1', 'house_ages2to5', 'house_ages6to12',
       'house_ages13to17', 'house_ages18plus', 'house_type', 'house_own'],
      dtype='object')

In [4]:
# these are interesting to read through :)
misc_causes = df[df["cause_recoded"] == "Other (Please specify)"]["cause_other_text"]
for el in misc_causes:
    print(el)

god
The reason is because GOD wants it
because the Bible says all this will come to pass

everything is happening for a purpose
according to the Bible we are coming to the end of this world as we know it now .
only God can effect what is happening in the world
The end of time is nearing.
God
POLITICAL

with the world so hate mongered the lord will soon comr to take us home

God is in control.
gases in the air
It's probably not just one thing and some facts and statistics are misused.
It is the fulfillment of the HOLY BIBLE. I belive there will come a day when "we wont be able to tell the differences in the seasons" as the bible reads.
God End of Days

global warming has been going since cave man and dinosaurs,and since god walked this earth
God said that the seasons would change.
is a way to control people and a new way to make money by force to buy what they are creating

happens every so often
LORD
The Government is controling the weather and causing the glacial melts
God's plan for 

In [5]:
vars_to_remove = [
    "case_ID", "wave", "weight_wave", "weight_aggregate",
    "cause_original", "cause_other_text", # cause_recoded summarizes
    "age", "educ", "income", # other vars bin into categories
    "ideology", "party", "party_w_leaners", # party_x_ideo summarizes
    "region4", # using region9 instead
    "religion_other_nonchristian", # diverse answers, not super important
    "house_ages0to1", "house_ages2to5", "house_ages6to12", "house_ages13to17", "house_ages18plus", # not sure how to deal with
]
df_filtered = df.drop(vars_to_remove, axis=1)
df_filtered.columns

Index(['year', 'happening', 'cause_recoded', 'sci_consensus', 'worry',
       'harm_personally', 'harm_US', 'harm_dev_countries', 'harm_future_gen',
       'harm_plants_animals', 'when_harm_US', 'reg_CO2_pollutant',
       'reg_utilities', 'fund_research', 'reg_coal_emissions', 'discuss_GW',
       'hear_GW_media', 'gender', 'age_category', 'generation',
       'educ_category', 'income_category', 'race', 'party_x_ideo',
       'registered_voter', 'region9', 'religion', 'evangelical',
       'service_attendance', 'marit_status', 'employment', 'house_head',
       'house_size', 'house_type', 'house_own'],
      dtype='object')

In [55]:
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder

# prepare a transaction list by converting each data point
# to ['attr1: val1', 'attr2: val2'] and creating a list of
# all data points


transaction_list = []

column_names = df_filtered.columns.to_numpy().astype(str)
colons = np.full(column_names.size, ": ")
prefixes = np.char.add(column_names, colons)

# df_filtered_sample = df_filtered.sample(n=10000, random_state=7)
df_filtered_sample = df_filtered

for i in range(len(df_filtered_sample)):
    values = df_filtered_sample.iloc[i].values.astype(str)
    items = np.char.add(prefixes, values)
    transaction_list.append(items)


# convert into a pandas dataframe with True/False for each possible item in each row
# the mlxtend library requires this format to do frequent pattern matching

encoder = TransactionEncoder()
encoded_array = encoder.fit(transaction_list).transform(transaction_list)
tdb = pd.DataFrame(encoded_array, columns = encoder.columns_)

In [56]:
freq_patterns = fpgrowth(tdb, min_support=0.2, use_colnames=True)
freq_patterns

,support,itemsets
0,0.850810,(registered_voter: Registered)
1,0.748208,(house_own: Owned by you or someone in your ho...
2,0.742302,"(race: White, Non-Hispanic)"
3,0.678192,(happening: Yes)
4,0.664853,(evangelical: No)
...,...,...
2074,0.214494,"(happening: Yes, reg_utilities: nan)"
2075,0.202383,"(evangelical: No, reg_utilities: nan)"
2076,0.226706,"(reg_utilities: nan, house_own: Owned by you o..."
2077,0.220534,"(race: White, Non-Hispanic, reg_utilities: nan)"


In [57]:
def filter_patterns(required_var):
    filter_col_name = f"contains_{required_var}"
    freq_patterns[filter_col_name] = freq_patterns["itemsets"].apply(lambda x: any(required_var in elem for elem in x))
    filtered_patterns = freq_patterns.loc[ freq_patterns[filter_col_name] ]
    filtered_patterns = filtered_patterns.sort_values(by=["support"], ascending=False).reset_index(drop=True)
    return filtered_patterns
    
filter_patterns("happening")

,support,itemsets,contains_happening
0,0.678192,(happening: Yes),True
1,0.583820,"(happening: Yes, registered_voter: Registered)",True
2,0.501493,"(house_head: Head of household, happening: Yes)",True
3,0.500033,"(happening: Yes, house_own: Owned by you or so...",True
4,0.493529,"(evangelical: No, happening: Yes)",True
...,...,...,...
790,0.200226,"(employment: Working - as a paid employee, reg...",True
791,0.200226,"(house_head: Head of household, happening: Yes...",True
792,0.200192,(sci_consensus: Most scientists think global w...,True
793,0.200159,"(harm_plants_animals: A great deal, happening:...",True


Questions:
* How to factor in the weights for pattern mining?

In [52]:
len(df)

30136